<a href="https://colab.research.google.com/github/prasu-baran/Data-Science/blob/main/Pokemon_Legendary_identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.getcwd()

'/content'

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [5]:
ls

drive/  sample_data/


In [6]:
pokemon_data = pd.read_csv('drive/MyDrive/Pokemon Data.csv')

In [7]:
poke = pd.DataFrame(pokemon_data)
poke.head()

,Number,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,...,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,Green,True,0.875,Monster,Grass,False,0.71,6.9,45,quadruped
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,Green,True,0.875,Monster,Grass,False,0.99,13.0,45,quadruped
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,Green,True,0.875,Monster,Grass,True,2.01,100.0,45,quadruped
3,4,Charmander,Fire,NaN,309,39,52,43,60,50,...,Red,True,0.875,Monster,Dragon,False,0.61,8.5,45,bipedal_tailed
4,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,...,Red,True,0.875,Monster,Dragon,False,1.09,19.0,45,bipedal_tailed


In [8]:
poke.isnull().sum()

,0
Number,0
Name,0
Type_1,0
Type_2,371
Total,0
HP,0
Attack,0
Defense,0
Sp_Atk,0
Sp_Def,0


In [9]:
poke.shape

(721, 23)

In [10]:
poke['Pr_Male'].value_counts()

,count
Pr_Male,
0.500,458
0.875,101
0.000,23
0.250,22
1.000,19
0.750,19
0.125,2


In [11]:
poke['Pr_Male'].fillna(0.500, inplace=True)
poke['Pr_Male'].isnull().sum()

<ipython-input-11-a65f76a45605>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  poke['Pr_Male'].fillna(0.500, inplace=True)


np.int64(0)

In [12]:
poke.tail()

,Number,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,...,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
716,717,Yveltal,Dark,Flying,680,126,131,95,131,98,...,Red,False,0.5,Undiscovered,NaN,False,5.79,203.0,45,two_wings
717,718,Zygarde,Dragon,Ground,600,108,100,121,81,95,...,Green,False,0.5,Undiscovered,NaN,False,5.00,305.0,3,serpentine_body
718,719,Diancie,Rock,Fairy,600,50,100,150,100,150,...,Pink,False,0.5,Undiscovered,NaN,True,0.71,8.8,3,head_arms
719,720,Hoopa,Psychic,Ghost,600,80,110,60,150,130,...,Purple,False,0.5,Undiscovered,NaN,False,0.51,9.0,3,head_only
720,721,Volcanion,Fire,Water,600,80,110,120,130,90,...,Brown,False,0.5,Undiscovered,NaN,False,1.70,195.0,3,quadruped


In [13]:
new_poke = poke.drop(['Type_2', 'Egg_Group_2'], axis=1)

In [14]:
new_poke.describe()

,Number,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Pr_Male,Height_m,Weight_kg,Catch_Rate
count,721.00000,721.000000,721.000000,721.000000,721.000000,721.000000,721.000000,721.000000,721.000000,721.000000,721.000000,721.000000,721.000000
mean,361.00000,417.945908,68.380028,75.013870,70.808599,68.737864,69.291262,65.714286,3.323162,0.547677,1.144979,56.773370,100.246879
std,208.27906,109.663671,25.848272,28.984475,29.296558,28.788005,27.015860,27.277920,1.669873,0.189693,1.044369,89.095667,76.573513
min,1.00000,180.000000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.000000,0.000000,0.100000,0.100000,3.000000
25%,181.00000,320.000000,50.000000,53.000000,50.000000,45.000000,50.000000,45.000000,2.000000,0.500000,0.610000,9.400000,45.000000
50%,361.00000,424.000000,65.000000,74.000000,65.000000,65.000000,65.000000,65.000000,3.000000,0.500000,0.990000,28.000000,65.000000
75%,541.00000,499.000000,80.000000,95.000000,85.000000,90.000000,85.000000,85.000000,5.000000,0.500000,1.400000,61.000000,180.000000
max,721.00000,720.000000,255.000000,165.000000,230.000000,154.000000,230.000000,160.000000,6.000000,1.000000,14.500000,950.000000,255.000000


In [16]:
new_poke['Type_1'].value_counts()

,count
Type_1,
Water,105
Normal,93
Grass,66
Bug,63
Fire,47
Psychic,47
Rock,41
Electric,36
Ground,30


In [17]:
new_poke['Generation'].value_counts()

,count
Generation,
5,156
1,151
3,135
4,107
2,100
6,72


In [18]:
new_poke['Color'].value_counts()

,count
Color,
Blue,134
Brown,110
Green,79
Red,75
Grey,69
Purple,65
Yellow,64
White,52
Pink,41


In [19]:
new_poke['Egg_Group_1'].value_counts()

,count
Egg_Group_1,
Field,169
Monster,74
Water_1,74
Undiscovered,73
Bug,66
Mineral,46
Flying,44
Amorphous,41
Human-Like,37


In [20]:
new_poke['Body_Style'].value_counts()

,count
Body_Style,
bipedal_tailed,158
quadruped,135
bipedal_tailless,109
two_wings,63
head_arms,39
head_only,34
with_fins,31
head_base,30
insectoid,30


In [21]:
poke_type1 = new_poke.replace(['Water', 'Ice'], 'Water')
poke_type1 = poke_type1.replace(['Grass', 'Bug'], 'Grass')
poke_type1 = poke_type1.replace(['Ground', 'Rock'], 'Rock')
poke_type1 = poke_type1.replace(['Psychic', 'Dark', 'Ghost', 'Fairy'], 'Dark')
poke_type1 = poke_type1.replace(['Electric', 'Steel'], 'Electric')

poke_type1['Type_1'].value_counts()

,count
Type_1,
Grass,129
Water,128
Dark,115
Normal,93
Rock,71
Electric,58
Fire,47
Poison,28
Fighting,25


In [22]:
poke_type1.head()

,Number,Name,Type_1,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,...,isLegendary,Color,hasGender,Pr_Male,Egg_Group_1,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
0,1,Bulbasaur,Grass,318,45,49,49,65,65,45,...,False,Green,True,0.875,Monster,False,0.71,6.9,45,quadruped
1,2,Ivysaur,Grass,405,60,62,63,80,80,60,...,False,Green,True,0.875,Monster,False,0.99,13.0,45,quadruped
2,3,Venusaur,Grass,525,80,82,83,100,100,80,...,False,Green,True,0.875,Monster,True,2.01,100.0,45,quadruped
3,4,Charmander,Fire,309,39,52,43,60,50,65,...,False,Red,True,0.875,Monster,False,0.61,8.5,45,bipedal_tailed
4,5,Charmeleon,Fire,405,58,64,58,80,65,80,...,False,Red,True,0.875,Monster,False,1.09,19.0,45,bipedal_tailed


In [23]:
ref1 = dict(poke_type1['Body_Style'].value_counts())

poke_type1['Body_Style_new'] = poke_type1['Body_Style'].map(ref1)

In [24]:
poke_type1['Body_Style_new'].head()

,Body_Style_new
0,135
1,135
2,135
3,158
4,158


In [25]:
poke_type1['Body_Style'].head()

,Body_Style
0,quadruped
1,quadruped
2,quadruped
3,bipedal_tailed
4,bipedal_tailed


In [26]:
types_poke = pd.get_dummies(poke_type1['Type_1'])
color_poke = pd.get_dummies(poke_type1['Color'])

X = pd.concat([poke_type1, types_poke], axis=1)
X = pd.concat([X, color_poke], axis=1)

X.head()

,Number,Name,Type_1,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,...,Black,Blue,Brown,Green,Grey,Pink,Purple,Red,White,Yellow
0,1,Bulbasaur,Grass,318,45,49,49,65,65,45,...,False,False,False,True,False,False,False,False,False,False
1,2,Ivysaur,Grass,405,60,62,63,80,80,60,...,False,False,False,True,False,False,False,False,False,False
2,3,Venusaur,Grass,525,80,82,83,100,100,80,...,False,False,False,True,False,False,False,False,False,False
3,4,Charmander,Fire,309,39,52,43,60,50,65,...,False,False,False,False,False,False,False,True,False,False
4,5,Charmeleon,Fire,405,58,64,58,80,65,80,...,False,False,False,False,False,False,False,True,False,False


In [27]:
X.columns

Index(['Number', 'Name', 'Type_1', 'Total', 'HP', 'Attack', 'Defense',
       'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary', 'Color',
       'hasGender', 'Pr_Male', 'Egg_Group_1', 'hasMegaEvolution', 'Height_m',
       'Weight_kg', 'Catch_Rate', 'Body_Style', 'Body_Style_new', 'Dark',
       'Dragon', 'Electric', 'Fighting', 'Fire', 'Flying', 'Grass', 'Normal',
       'Poison', 'Rock', 'Water', 'Black', 'Blue', 'Brown', 'Green', 'Grey',
       'Pink', 'Purple', 'Red', 'White', 'Yellow'],
      dtype='object')

In [28]:
X_ = X.drop(['Number', 'Name', 'Type_1', 'Color', 'Egg_Group_1'], axis = 1)
X_.shape

(721, 38)

In [29]:
X.shape

(721, 43)

In [30]:
y = X_['isLegendary']
X_final = X_.drop(['isLegendary', 'Body_Style'], axis = 1)

In [31]:
X_final.columns

Index(['Total', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed',
       'Generation', 'hasGender', 'Pr_Male', 'hasMegaEvolution', 'Height_m',
       'Weight_kg', 'Catch_Rate', 'Body_Style_new', 'Dark', 'Dragon',
       'Electric', 'Fighting', 'Fire', 'Flying', 'Grass', 'Normal', 'Poison',
       'Rock', 'Water', 'Black', 'Blue', 'Brown', 'Green', 'Grey', 'Pink',
       'Purple', 'Red', 'White', 'Yellow'],
      dtype='object')

In [32]:
X_final.head()

,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,hasGender,Pr_Male,...,Black,Blue,Brown,Green,Grey,Pink,Purple,Red,White,Yellow
0,318,45,49,49,65,65,45,1,True,0.875,...,False,False,False,True,False,False,False,False,False,False
1,405,60,62,63,80,80,60,1,True,0.875,...,False,False,False,True,False,False,False,False,False,False
2,525,80,82,83,100,100,80,1,True,0.875,...,False,False,False,True,False,False,False,False,False,False
3,309,39,52,43,60,50,65,1,True,0.875,...,False,False,False,False,False,False,False,True,False,False
4,405,58,64,58,80,65,80,1,True,0.875,...,False,False,False,False,False,False,False,True,False,False


In [33]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_final, y, test_size=0.2)

In [34]:
random_model = RandomForestClassifier(n_estimators=500, random_state = 42)

In [35]:
#Fit
model_final = random_model.fit(Xtrain, ytrain)

y_pred = model_final.predict(Xtest)

#Checking the accuracy
random_model_accuracy = round(model_final.score(Xtrain, ytrain)*100,2)
print(round(random_model_accuracy, 2), '%')

100.0 %


In [36]:
random_model_accuracy1 = round(random_model.score(Xtest, ytest)*100,2)
print(round(random_model_accuracy1, 2), '%')

97.93 %


In [37]:
print(type(ytest))
Ytest = np.array(ytest)

<class 'pandas.core.series.Series'>


In [38]:
type(y_pred)

numpy.ndarray

In [40]:
count = 0
for i in range(len(ytest)):
  if Ytest[i] == y_pred[i]:
    count = count + 1

print((count/len(Ytest))*100)

97.93103448275862
